In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV

# 你的MLP类和数据加载类
from src.models import MLP, KAN
from src.data import HARDataSet

In [2]:
params = {
    "module__layers_hidden": [
        [561, 128, 64, 6],
        [561, 100, 50, 6],
        [561, 64, 32, 6],
    ],
}

# 使用你的KAN结构
net = NeuralNetClassifier(
    module=KAN,
    module__grid_size=5,
    module__spline_order=3,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    iterator_train__shuffle=True,
    lr=0.001,
    batch_size=128,
    max_epochs=30,
    verbose=0,  # 或者更高的verbose级别以查看训练过程
)

# 初始化 GridSearchCV
gs = GridSearchCV(net, params, refit=True, cv=3, scoring="accuracy", verbose=2)

In [3]:
# 加载数据
X_train_file = "dataset/HAR/train/X_train.txt"
y_train_file = "dataset/HAR/train/y_train.txt"

train_dataset = HARDataSet(X_train_file, y_train_file)
X_train = train_dataset.X.numpy()
y_train = train_dataset.y.numpy()

# 运行网格搜索
gs.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END ............module__layers_hidden=[561, 128, 64, 6]; total time=  40.3s
[CV] END ............module__layers_hidden=[561, 128, 64, 6]; total time=  33.6s
[CV] END ............module__layers_hidden=[561, 128, 64, 6]; total time=  36.2s
[CV] END ............module__layers_hidden=[561, 100, 50, 6]; total time=  37.0s
[CV] END ............module__layers_hidden=[561, 100, 50, 6]; total time=  37.3s
[CV] END ............module__layers_hidden=[561, 100, 50, 6]; total time=  36.9s
[CV] END .............module__layers_hidden=[561, 64, 32, 6]; total time=  35.0s
[CV] END .............module__layers_hidden=[561, 64, 32, 6]; total time=  32.2s
[CV] END .............module__layers_hidden=[561, 64, 32, 6]; total time=  29.2s


GridSearchCV(cv=3,
             estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class 'src.models.kan.KAN'>,
  module__grid_size=5,
  module__spline_order=3,
),
             param_grid={'module__layers_hidden': [[561, 128, 64, 6],
                                                   [561, 100, 50, 6],
                                                   [561, 64, 32, 6]]},
             scoring='accuracy', verbose=2)

In [4]:
print("Best parameters:", gs.best_params_)
print("Best cross-validation score: {:.4f}".format(gs.best_score_))

Best parameters: {'module__layers_hidden': [561, 128, 64, 6]}
Best cross-validation score: 0.9369


In [5]:
params = {
    "module__hidden_layers": [
        [512, 128],
        [512, 256],
        [512, 64],
    ],  # 不同的隐藏层配置
}

# 使用你的MLP结构
net = NeuralNetClassifier(
    module=MLP,
    module__input_size=561,
    module__output_size=6,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    iterator_train__shuffle=True,
    batch_size=128,
    lr=0.001,
    max_epochs=30,  # 固定训练轮次为30
    verbose=0,  # 或者更高的verbose级别以查看训练过程
)

# 初始化 GridSearchCV
gs = GridSearchCV(net, params, refit=True, cv=3, scoring="accuracy", verbose=2)

In [6]:
# 加载数据
X_train_file = "dataset/HAR/train/X_train.txt"
y_train_file = "dataset/HAR/train/y_train.txt"

train_dataset = HARDataSet(X_train_file, y_train_file)
X_train = train_dataset.X.numpy()
y_train = train_dataset.y.numpy()

# 运行网格搜索
gs.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END ...................module__hidden_layers=[512, 128]; total time=   7.5s
[CV] END ...................module__hidden_layers=[512, 128]; total time=   6.6s
[CV] END ...................module__hidden_layers=[512, 128]; total time=   6.4s
[CV] END ...................module__hidden_layers=[512, 256]; total time=   7.6s
[CV] END ...................module__hidden_layers=[512, 256]; total time=   7.5s
[CV] END ...................module__hidden_layers=[512, 256]; total time=   7.2s
[CV] END ....................module__hidden_layers=[512, 64]; total time=   6.5s
[CV] END ....................module__hidden_layers=[512, 64]; total time=   7.1s
[CV] END ....................module__hidden_layers=[512, 64]; total time=   7.6s


GridSearchCV(cv=3,
             estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class 'src.models.mlp.MLP'>,
  module__input_size=561,
  module__output_size=6,
),
             param_grid={'module__hidden_layers': [[512, 128], [512, 256],
                                                   [512, 64]]},
             scoring='accuracy', verbose=2)

In [7]:
print("Best parameters:", gs.best_params_)
print("Best cross-validation score: {:.4f}".format(gs.best_score_))

Best parameters: {'module__hidden_layers': [512, 128]}
Best cross-validation score: 0.9321
